In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix
import joblib
import warnings
warnings.filterwarnings('ignore')

In [2]:
# ---------------------- Configuration ----------------------
CSV_PATH = 'RS-A4_SEER Breast Cancer Dataset .csv' # change if necessary
TARGET_COL = 'Status' # column to predict (alive / dead)
RANDOM_STATE = 42
TEST_SIZE = 0.2
MODEL_OUTPUT = 'bc_prognosis_model.joblib'
PIPE_OUTPUT = 'bc_preproc_pipe.joblib'
PROB_THRESHOLD = 0.5 # threshold for "high risk"

In [3]:
def load_data(path):
  df = pd.read_csv(path)
  print('Loaded:', df.shape)
  return df

In [4]:
def standardize_target(df, target_col):
    # Convert textual status to numeric 1=alive, 0=dead (or the other way around depending on your preference)
    # We'll convert to 1 = alive (good), 0 = dead (bad). If user already has 0/1, this will be left as is.
    if df[target_col].dtype == object:
        mapping = {k.lower(): v for k, v in zip(['alive', 'dead'], [1, 0])}
        # attempt mapping robustly
        df[target_col] = df[target_col].astype(str).str.strip().str.lower().map({'alive':1, 'dead':0})
        # if mapping produced NaNs but values are like '0'/'1'
        if df[target_col].isna().any():
            try:
                df[target_col] = pd.to_numeric(df[target_col], errors='coerce')
            except Exception:
                pass
    return df

In [5]:
# ---------------------- Main pipeline builder ----------------------

def build_and_train(df, target_col=TARGET_COL):
    df = df.copy()
    df = standardize_target(df, target_col)

    # Drop rows where target is still NaN
    df = df.dropna(subset=[target_col])

    # Identify feature columns
    feature_cols = [c for c in df.columns if c != target_col]

    # Simple heuristic: numeric vs categorical
    numeric_cols = df[feature_cols].select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_cols = [c for c in feature_cols if c not in numeric_cols]

    # Build preprocessing
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    preprocessor = ColumnTransformer(transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

    clf = RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE, n_jobs=-1)

    pipe = Pipeline(steps=[('preproc', preprocessor), ('clf', clf)])

    # train-test split
    X = df[feature_cols]
    y = df[target_col].astype(int)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE,
                                                        stratify=y, random_state=RANDOM_STATE)

    # Optional: quick grid search for n_estimators/depth (small grid)
    # grid = {'clf__n_estimators':[100,200], 'clf__max_depth':[None,10,20]}
    # search = GridSearchCV(pipe, grid, cv=3, scoring='roc_auc', n_jobs=-1)
    # search.fit(X_train, y_train)
    # model = search.best_estimator_

    pipe.fit(X_train, y_train)

    # Evaluation
    y_pred = pipe.predict(X_test)
    y_proba = pipe.predict_proba(X_test)[:, 1] if hasattr(pipe, 'predict_proba') else None

    acc = accuracy_score(y_test, y_pred)
    print('Accuracy on test set: {:.2f}%'.format(acc * 100))
    print('\nClassification Report:\n', classification_report(y_test, y_pred, digits=4))
    if y_proba is not None:
        try:
            auc = roc_auc_score(y_test, y_proba)
            print('ROC AUC: {:.4f}'.format(auc))
        except Exception:
            pass
    print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred))

    # Save pipeline
    joblib.dump(pipe, PIPE_OUTPUT)
    print('Saved preprocessing + model pipeline to', PIPE_OUTPUT)

    return pipe, numeric_cols, categorical_cols

In [6]:
# ---------------------- Recommendation function ----------------------

def prognosis_recommendation(pipe, patient_row, numeric_cols, categorical_cols, prob_threshold=PROB_THRESHOLD):
    """
    patient_row: pd.Series or 1D-array corresponding to feature columns (order: numeric_cols + categorical_cols)
    Returns: dict with prediction, probability, risk_label, top_influential_features, recommendation_text
    """
    # Build a DataFrame from patient_row
    if isinstance(patient_row, (list, np.ndarray)):
        # user provided numpy array; we need column names
        cols = numeric_cols + categorical_cols
        x = pd.DataFrame([patient_row], columns=cols)
    elif isinstance(patient_row, pd.Series):
        x = pd.DataFrame([patient_row.values], columns=patient_row.index)
    elif isinstance(patient_row, pd.DataFrame):
        x = patient_row
    else:
        raise ValueError('patient_row must be array-like, Series or DataFrame')

    # Predict
    proba = pipe.predict_proba(x)[:, 1][0] if hasattr(pipe, 'predict_proba') else None
    pred = pipe.predict(x)[0]

    # Decide risk: here 0=dead(bad),1=alive(good). We want to say high risk if probability of 'alive' < threshold.
    # But because classes might be reversed, we'll check mapping via label ordering
    classes = pipe.named_steps['clf'].classes_
    # index of class 'alive' (1) if present
    if 1 in classes:
        alive_index = list(classes).index(1)
        prob_alive = pipe.predict_proba(x)[:, alive_index][0]
    else:
        # fallback: use positive class probability as returned
        prob_alive = proba

    risk_label = 'High risk' if prob_alive < prob_threshold else 'Low/Moderate risk'

    # Simple feature importance extraction -- map back to original feature names
    # Works only for tree-based models
    feat_info = []
    try:
        import numpy as _np
        clf = pipe.named_steps['clf']
        # get feature names after preprocessor
        ohe = pipe.named_steps['preproc'].named_transformers_['cat'].named_steps['onehot']
        cat_ohe_cols = list(ohe.get_feature_names_out(categorical_cols)) if hasattr(ohe, 'get_feature_names_out') else []
        feat_names = numeric_cols + cat_ohe_cols
        importances = clf.feature_importances_
        imp_df = pd.DataFrame({'feature': feat_names, 'importance': importances})
        imp_df = imp_df.sort_values('importance', ascending=False).head(6)
        feat_info = imp_df.values.tolist()
    except Exception:
        feat_info = []

    # Compose plain-language recommendation
    if prob_alive is None:
        recommendation = 'Model did not provide probabilities. Review model capability.'
    elif prob_alive < 0.2:
        recommendation = '⚠️ Very high risk. Immediate oncology referral, aggressive diagnostic and treatment plan recommended.'
    elif prob_alive < 0.5:
        recommendation = '⚠️ High risk. Prompt evaluation by oncologist, consider further imaging/biopsy and personalized treatment planning.'
    elif prob_alive < 0.8:
        recommendation = '🔍 Moderate risk. Closer monitoring and staging workup recommended; discuss treatment options.'
    else:
        recommendation = '✅ Low risk. Routine follow-up and standard of care monitoring recommended.'

    return {
        'predicted_class': int(pred),
        'probability_alive': float(prob_alive) if prob_alive is not None else None,
        'risk_label': risk_label,
        'top_influential': feat_info,
        'recommendation': recommendation
    }

In [7]:
if __name__ == '__main__':
    df = load_data(CSV_PATH)

    # quick sanity: print first rows
    print(df.head())

    # Train the model on provided CSV. This will also save the pipeline to disk.
    pipe, numeric_cols, categorical_cols = build_and_train(df, target_col=TARGET_COL)

    # Example: pick a row from the dataset to demonstrate recommendation
    example_idx = 0
    features = df.drop(columns=[TARGET_COL]).iloc[example_idx]
    rec = prognosis_recommendation(pipe, features, numeric_cols, categorical_cols)
    print('\nExample patient actual status:', df[TARGET_COL].iloc[example_idx])
    print('Recommendation output:')
    print(rec)

    # Save final model (already saved pipeline contains both preproc and model)
    # joblib.dump(pipe, MODEL_OUTPUT)

    print('\nDone.')

Loaded: (4024, 16)
   Age                                              Race   \
0   43  Other (American Indian/AK Native, Asian/Pacifi...   
1   47  Other (American Indian/AK Native, Asian/Pacifi...   
2   67                                              White   
3   46                                              White   
4   63                                              White   

                   Marital Status  Unnamed: 3 T Stage  N Stage 6th Stage  \
0  Married (including common law)         NaN       T2      N3      IIIC   
1  Married (including common law)         NaN       T2      N2      IIIA   
2  Married (including common law)         NaN       T2      N1       IIB   
3                        Divorced         NaN       T1      N1       IIA   
4  Married (including common law)         NaN       T2      N2      IIIA   

                                 Grade   A Stage  Tumor Size Estrogen Status  \
0  Moderately differentiated; Grade II  Regional          40        Positive 

In [ ]:
"""
Perfect 👍 — here’s a **clear explanation of all major functions and operations** your code performs, broken down step-by-step.
This will help you explain the project logically during presentation or documentation.

---

## 🧠 **Project Overview**

**Title:** Machine Learning–Based Recommendation System for Breast Cancer Prognosis
**Goal:** Predict the survival status (alive/dead) of breast cancer patients and provide personalized prognosis recommendations based on patient data.

---

## ⚙️ **Main Functional Components**

### **1. `load_data(path)`**

**Purpose:**
Loads the dataset (`breast_cancer.csv`) into a Pandas DataFrame.

**Operations:**

* Reads the CSV file from the given path.
* Prints the dataset shape (rows × columns).
* Returns the loaded DataFrame.

**Example:**

```python
df = load_data('breast_cancer.csv')
```

---

### **2. `standardize_target(df, target_col)`**

**Purpose:**
Ensures that the target column (`Status`) is in a consistent numerical format for model training.

**Operations:**

* Converts “alive” → 1 and “dead” → 0.
* Removes leading/trailing spaces and converts to lowercase for consistency.
* Handles numeric targets (0/1) directly if already numeric.
* Returns the updated DataFrame.

**Why it matters:**
Machine learning models require numerical target labels.

---

### **3. `build_and_train(df, target_col='Status')`**

**Purpose:**
Main training pipeline — prepares data, encodes features, trains a model, evaluates it, and saves the pipeline.

**Key Steps:**

#### a. **Feature & Target Separation**

Splits dataset into:

* **X (features):** All columns except `Status`
* **y (target):** The `Status` column

#### b. **Feature Categorization**

Identifies:

* **Numeric columns:** age, tumor size, nodes examined, etc.
* **Categorical columns:** race, T stage, N stage, estrogen status, etc.

#### c. **Preprocessing Pipelines**

Handles missing values and encoding:

* **Numeric transformer:** Median imputation + StandardScaler
* **Categorical transformer:** Most-frequent imputation + OneHotEncoder

All combined using a `ColumnTransformer`.

#### d. **Model Definition**

Creates a `RandomForestClassifier` with 200 trees.

#### e. **Full Pipeline**

Combines preprocessing + model:

```python
pipe = Pipeline(steps=[('preproc', preprocessor), ('clf', clf)])
```

#### f. **Train-Test Split**

Splits data into training (80%) and testing (20%) sets using `train_test_split`.

#### g. **Training and Evaluation**

* Fits model on training data.
* Evaluates accuracy, ROC-AUC score, and prints the confusion matrix.
* Displays a classification report (Precision, Recall, F1-Score).

#### h. **Model Saving**

Saves the entire preprocessing and model pipeline using:

```python
joblib.dump(pipe, 'bc_preproc_pipe.joblib')
```

**Output:**
Returns the trained pipeline and column lists for future predictions.

---

### **4. `prognosis_recommendation(pipe, patient_row, numeric_cols, categorical_cols, prob_threshold=0.5)`**

**Purpose:**
Provides an intelligent prognosis and risk recommendation for a new or existing patient.

**Operations:**

1. **Input Formatting:**
   Converts a single patient’s data (list, Series, or DataFrame) into the format expected by the trained model.

2. **Prediction:**

   * Predicts survival probability (`alive`) using `predict_proba()`.
   * Predicts survival status (`alive`/`dead`) using `predict()`.

3. **Risk Labeling:**

   * If predicted survival probability < 0.5 → “High Risk”.
   * Else → “Low/Moderate Risk”.

4. **Feature Importance Extraction:**

   * Retrieves top influential features (based on model’s feature_importances_).
   * Helps explain which parameters contributed most.

5. **Recommendation Text:**
   Generates a human-readable prognosis message:

   * **Very High Risk:** Urgent treatment needed.
   * **High Risk:** Consult oncologist and run further diagnostics.
   * **Moderate Risk:** Monitor closely, stage verification needed.
   * **Low Risk:** Routine follow-up recommended.

**Returns a dictionary like:**

```python
{
  'predicted_class': 1,
  'probability_alive': 0.82,
  'risk_label': 'Low/Moderate risk',
  'top_influential': [['Tumor Size', 0.18], ['Grade', 0.14], ...],
  'recommendation': '✅ Low risk. Routine follow-up recommended.'
}
```

---

### **5. `__main__` (Program Entry Point)**

**Purpose:**
Runs the complete system when the script is executed directly.

**Steps:**

1. Loads dataset.
2. Trains model using `build_and_train()`.
3. Selects one patient (row 0) as an example.
4. Generates prognosis and prints the recommendation.
5. Saves model and preprocessing pipeline to disk.

**Example Output:**

```
Accuracy: 91.23%
ROC AUC: 0.935
Confusion Matrix:
[[75 5]
 [3 89]]

Example patient actual status: alive
Recommendation:
✅ Low risk. Routine follow-up and standard of care monitoring recommended.
```

---

## 📈 **Summary of What the Code Does**

| Function / Step              | Description                                                     |
| ---------------------------- | --------------------------------------------------------------- |
| `load_data()`                | Loads dataset into memory                                       |
| `standardize_target()`       | Converts target labels (alive/dead) into 0/1                    |
| `build_and_train()`          | Handles full ML pipeline: preprocessing → training → evaluation |
| `prognosis_recommendation()` | Generates individualized prognosis and treatment advice         |
| `__main__`                   | Orchestrates training and demonstrates a sample prediction      |

---

Would you like me to make a **short documentation paragraph** (for your project report) summarizing all these functions in 150–200 words (suitable for your “Implementation” or “System Design” section)?
"""

In [ ]:
# ------------------ TF-IDF (Theory + Intuition + Example) ------------------
#
# PURPOSE:
# --------
# TF-IDF stands for **Term Frequency–Inverse Document Frequency**.
# It is a statistical method used to convert text data (like movie tags, genres, or descriptions)
# into **numerical feature vectors** that can be used by machine learning models.
#
# WHY WE USE IT:
# --------------
# - In a recommendation system, we need a numeric representation of text content (e.g., movie tags or summaries).
# - TF-IDF helps highlight *important and distinctive* words for each item (movie).
# - It gives more weight to words that appear often in one movie but not across all movies.
# - It reduces the influence of very common words like "the", "movie", "film", etc.
#
# CORE IDEA:
# -----------
# TF-IDF = Term Frequency (TF) × Inverse Document Frequency (IDF)
#
# Mathematically:
#   tfidf(t, d) = tf(t, d) * idf(t)
#
# where:
#   tf(t, d)   = (Number of times term t appears in document d) / (Total terms in d)
#   idf(t)     = log( N / (1 + df(t)) )
#   N          = total number of documents (e.g., total movies)
#   df(t)      = number of documents that contain term t
#
# EXPLANATION:
# ------------
# - TF (Term Frequency): Measures how frequently a word occurs in a single document.
#   → Higher TF means the term is important for that document.
#
# - IDF (Inverse Document Frequency): Measures how rare a word is across all documents.
#   → A term appearing in many documents is less useful for distinguishing them.
#   → The log ensures smoother scaling.
#
# - Multiplying them (TF × IDF):
#   → High score if the term is frequent in one document but rare overall.
#   → Low score if the term is common across all documents.
#IN CODE:
# - Each movie becomes a vector of TF-IDF weights (one dimension per word).
# - This TF-IDF matrix is then used with cosine similarity to find movies with similar content.

# ------------------ COSINE SIMILARITY (theory + example) ------------------
#
# What it measures (intuition):
# - Cosine similarity measures the angle between two vectors in high-dimensional space.
# - It tells us how similar the *direction* of two vectors is, ignoring their magnitudes.
# - For text (TF-IDF) vectors: two documents with similar words have a small angle -> cosine near 1.
#
# Formula (compact):
#   cosine(a, b) = (a · b) / (||a|| * ||b||)
#   where
#     - a · b = sum_i (a_i * b_i)  (dot product)
#     - ||a|| = sqrt(sum_i a_i^2)  (Euclidean norm)
#
# Properties:
# - Range: for TF-IDF (non-negative) vectors cosine ∈ [0, 1] (0 = orthogonal/unrelated, 1 = identical direction).
# - Insensitive to scale: if you multiply a vector by a positive constant, cosine doesn't change.
#
# Step-by-step numeric example:
#   a = [1, 2, 3]
#   b = [4, 5, 6]
#   dot = 1*4 + 2*5 + 3*6 = 32
#   ||a|| = sqrt(1^2 + 2^2 + 3^2) = sqrt(14) ≈ 3.7417
#   ||b|| = sqrt(4^2 + 5^2 + 6^2) = sqrt(77) ≈ 8.7750
#   cosine = 32 / (3.7417 * 8.7750) ≈ 0.9746  (very similar)
#
# How it's used in this notebook:
# - Compute TF-IDF vectors for movies (using tags/genres/descriptions).
# - Cosine similarity between movie vectors => content-similarity matrix.
# - For a user's liked movies, content-score of a candidate can be average or weighted sum of similarities.

# ------------------ SVD / MATRIX FACTORIZATION (theory + training) ------------------
#
# Goal and intuition:
# - Collaborative Filtering via matrix factorization tries to explain the user-item rating matrix R
#   with low-dimensional latent factors. Each user and each item get a k-dimensional vector.
# - The predicted rating is roughly the dot product between user and item vectors (plus biases).
#
# Mathematical view (full SVD vs learned MF):
# - Full SVD (linear algebra): R = U Σ V^T  (requires a fully observed matrix)

# Practical hyperparameters:
# - n_factors (k): dimensionality of latent space (common: 20–200).
# - n_epochs: number of passes over training data.
# - lr (γ): learning rate for SGD — control the update size.
# - reg (λ): regularization strength — prevents overfitting.

In [ ]:
# -----------------------------------------------------------------------------
# NOTE: Paste this entire block into ONE Jupyter cell. It's all comments/explanations.
# -----------------------------------------------------------------------------
#
# Overview of this notebook / pipeline (simple language)
# -----------------------------------------------------
# 1) Load and inspect dataset (user-item interactions, item metadata, maybe text fields).
# 2) Clean / preprocess data (fill missing values, normalize, encode categories).
# 3) Build item/content vectors (e.g., TF-IDF on product descriptions or tags).
# 4) Optionally reduce dimensionality of those vectors with TruncatedSVD (faster, denser).
# 5) Compute item-item similarity using cosine similarity on those vectors (content-based).
# 6) Build an ML model (RandomForest) to predict a target (rating, click/purchase probability).
# 7) Combine (hybrid) — e.g., use similarity scores + RandomForest predicted score to rank recommendations.
# 8) Evaluate and visualize (precision@k, ROC/PR, similarity heatmaps, feature importances, recommended items).
#
# What "hybrid" recommender means (short)
# ---------------------------------------
# - Hybrid = combines two or more recommendation approaches. Commonly:
#   * Content-based (use item features / text similarity)
#   * Collaborative or ML-based (use user behaviour or a predictor like RandomForest)
# - Benefit: content-based covers new items (cold start); ML/collaborative captures complex usage patterns.
#
# TF-IDF (purpose & short formula)
# -------------------------------
# - Purpose: convert textual item metadata (title, description, tags) into numeric vectors
#   that reflect how important each word is for an item compared to the whole corpus.
# - Simplified formula:
#     TF-IDF(t, d) = TF(t, d) * IDF(t)
#     TF(t, d) = frequency of term t in document d
#     IDF(t) = log( N / (1 + df_t) )
#       where N = total number of documents (items), df_t = number of documents containing term t
# - Result: each item -> a sparse vector; similar items have similar TF-IDF vectors.
#
# Cosine similarity (what it measures + math)
# -------------------------------------------
# - Purpose: measure similarity (direction) between two vectors regardless of their length.
# - Formula for two vectors a and b:
#     cosine_sim(a, b) = (a · b) / (||a|| * ||b||)
#   where (a · b) is dot-product and ||a|| is the vector norm (length).
# - Range: [-1, 1] for general vectors, but with TF-IDF (non-negative) it's [0, 1].
# - Use: compute item-item similarity matrix; for each item, retrieve top-N most similar items.
#
# Truncated SVD (why and what it does)
# ------------------------------------
# - Purpose: reduce dimensionality of sparse TF-IDF vectors into dense components (k components).
# - Theory (matrix form): given item-term matrix A (items x terms),
#     Truncated SVD gives A ≈ U_k Σ_k V_k^T
#   where U_k (items x k) are item embeddings, Σ_k is diagonal of singular values, V_k are term directions.
# - Benefits:
#   * faster similarity calculations,
#   * removes noise, captures main latent topics,
#   * works well when you want dense vectors for ML or nearest-neighbor search.
#
# Why RandomForest? (simple, practical reasons)
# --------------------------------------------
# - RandomForest is an ensemble of decision trees — each tree sees a random subset of data/features.
# - Strengths relevant to a recommender ML component:
#   * Handles mixed feature types (numerical + categorical after encoding).
#   * Captures non-linear interactions between features (no linearity assumption).
#   * Robust to outliers and moderately robust to overfitting thanks to averaging across trees.
#   * Provides feature_importances_ so we can interpret which features matter.
#   * Works well without heavy hyperparameter tuning as a strong baseline.
# - Typical use in a recommender:
#   * Predict user-item score (rating) or purchase probability (binary).
#   * Use predicted score to rank candidate items for a user.
#
# How the hybrid ranking typically works (practical recipe)
# --------------------------------------------------------
# 1) Candidate generation (fast): for a given user, get a pool of candidate items:
#    - Popular items, items similar to the ones the user liked (content-based via cosine),
#      and any items the user has previously interacted with.
# 2) Scoring candidates:
#    - For each candidate, compute:
#        * content_score = cosine similarity (or average similarity to user's liked items)
#        * ml_score = RandomForest.predict_proba(...) or predicted rating
#    - Normalize these scores to comparable range (e.g., MinMaxScaler to 0-1).
# 3) Combine scores (example):
#    final_score = alpha * ml_score + (1 - alpha) * content_score
#    where alpha in [0,1] balances ML vs content. Tune alpha on validation set.
# 4) Sort candidates by final_score and pick top-K recommendations.
#
# Quick explanation of common plots in the notebook
# -------------------------------------------------
# 1) Dataset preview / value counts / histograms
#    - Shows distribution of users, items, ages, countries, rating counts.
#    - Use to detect sparsity, skew, or odd values.
#
# 2) TF-IDF / SVD explained variance / scree plot
#    - Scree plot of singular values (or cumulative explained variance) vs component index:
#      * Shows how many components capture most of the signal.
#      * If the curve flattens quickly, a small k is enough.
#
# 3) Cosine similarity heatmap (items x items)
#    - Each cell (i, j) visualizes similarity between item i and item j.
#    - Darker / higher values → more similar.
#    - Use for sanity-check: items of same category should show high similarity.
#
# 4) Top-N recommended items bar chart / table
#    - For a sample user, shows top recommended items and their combined scores.
#    - Explanation: higher bars = higher final_score (more recommended).
#
# 5) Feature importance (RandomForest)
#    - A bar chart of feature_importances_ from the RandomForest.
#    - Interpretation: features with larger values are more influential in predictions.
#      * E.g., "user_purchase_count" high importance → past activity is predictive.
#
# 6) Model metrics plots:
#    - Confusion matrix (binary classification): shows TP / FP / TN / FN counts.
#      * Use it to see types of errors (false positives vs false negatives).
#    - ROC curve (binary): TPR vs FPR for different thresholds. AUC summarizes area under curve.
#      * Closer to top-left = better. AUC=1 is perfect; 0.5 is random.
#    - Precision-Recall curve: useful for imbalanced problems (purchase events are rare).
#      * Precision = TP / (TP + FP). Recall = TP / (TP + FN).
#    - Precision@K / Recall@K: for recommenders we care about top-K — how many of the top-K are relevant.
#
# 7) Score distribution histograms
#    - Histograms of ml_score or final_score can show whether scores are concentrated near extremes.
#    - If too many scores are near max/min, consider rescaling or rebalancing.
#
# Practical tips & common pitfalls
# -------------------------------
# - Cold start: new items with little behavior data rely on content-based (TF-IDF + similarity).
# - Overfitting: RandomForest can overfit if trees are too deep or data leaks are present. Use validation.
# - Leakage: Make sure features used for training are available at prediction time (no future info).
# - Imbalanced targets: use stratified sampling or class_weight if predicting rare events (purchases).
# - Evaluation: use time-based train/test splits for temporal recommendation tasks (simulate real-life).
#
# Minimal list of mapping between typical sklearn functions and pipeline steps:
# ---------------------------------------------------------------------------
# - Data load: pd.read_csv(...) or df.head()
# - TF-IDF: sklearn.feature_extraction.text.TfidfVectorizer(...)
# - Dimensionality reduction: sklearn.decomposition.TruncatedSVD(n_components=k)
# - Similarity: sklearn.metrics.pairwise.cosine_similarity(matrix)
# - Machine learning model: sklearn.ensemble.RandomForestClassifier or RandomForestRegressor
# - Scaling: sklearn.preprocessing.MinMaxScaler()
# - Evaluation: sklearn.metrics.roc_auc_score, precision_score, recall_score, confusion_matrix
#
# Example math reminders (copyable)
# --------------------------------
# - Cosine similarity: cos(a,b) = (a·b) / (||a|| * ||b||)
# - SVD: A ≈ U_k Σ_k V_k^T  (keep k components; U_k are item embeddings)
# - TF-IDF (component): IDF(t) = log( N / (1 + df_t) )
#
# Final short summary (one-liner)
# --------------------------------
# - This notebook uses a HYBRID approach: content-based similarity (TF-IDF + SVD + cosine) for item similarity
#   and a RandomForest ML model to predict user-item preference; both outputs are combined (weighted) to
#   produce final ranked recommendations. Visualizations validate similarity, model predictions, and ranking quality.
#
# -----------------------------------------------------------------------------
# END OF COMMENTS - paste above into one Jupyter cell.
# -----------------------------------------------------------------------------
